##  LSF版测试

使用软件中提供的LSF集群脚本 `cd 仓库/LSF`

### 安装完成后目录结构

```bash
tree -L 1
.
├── hg19_chromsome.bed #染色体长度信息
├── README.md
├── references  #参考基因文件夹
├── restriction_sites  #酶切文件夹
├── scripts  #LSF脚本文件夹
└── test  ##Hi-C 原始数据文件夹
```
在`test/fastq`文件夹中存放原始的测序数据
在`scripts`文件夹中存放LSF脚本

#### 配置GPU环境
由于软件需要在GPU节点进行计算，需要使用到cuda软件；首先配置自己的环境变量
```bash
cuda='/usr/local/cuda/'
export PATH=${cuda}"bin:"$PATH
export LD_LIBRARY_PATH=${cuda}"lib:"${LD_LIBRARY_PATH}
```

#### 1.1 HiCCUPS

+ `test/aligned/inter_30.hic`文件是使用juicer计算得到的
+ scripts目录是对应`LSF`版本的软连接
+ 指定计算节点为GPU节点`gpu`

##### 1.1.2提交任务

+ 单分辨率运行

```bash
bsub -q gpu -J juicer -n 10 -e juicer.err -o juicer.out -R span[hosts=1]  "java -jar scripts/juicer_tools.jar  hiccups -m 512 -r 10000 -f 0.1 -p 2  -i 5 -d 20000 -c 1  -k KR --threads 10 --ignore-sparsity  test/aligned/inter_30.hic ./chrosomeLoop"
```


+ 多分辨率运行

```bash
bsub -q gpu -J juicer -n 10 -e juicer.err -o juicer.out -R span[hosts=1]  "java -jar scripts/juicer_tools.jar hiccups -m 512 -r 5000,10000 -f 0.1,0.1 -p 4,2 -i 7,5 -d 20000,20000 -c 1 -k KR --threads 10 --ignore-sparsity test/aligned/inter_30.hic ./chrosomeLoop"
```

##### 1.1.3 输出结果
在单个分辨率下的计算时，在输出目录中仅仅只包含单个文件
`fdr_thresholds_10000`
多分辨率情况下输出结果
```bash
.
├── fdr_thresholds_10000
└── fdr_thresholds_5000
```

#### 1.2Eigenvector
划分compartments

+ 划分chr1号染色体在1M的分辨率下，得分
```bash
java -jar scripts/juicer_tools.jar eigenvector KR test/aligned/inter_30.hic chr1  BP 1000000 >111
```
结果
+ 每一行代表1M的区间的得分
```bash
0.010239605229520426
-0.0726968084857949
-0.04291854477085067
-0.029418479599141876
-0.004632664168007863
0.015414336275401206
-0.08943360513489482
-0.06504310611117355
```



